In [2]:
import pandas as pd
from decimal import Decimal
from datetime import timedelta, date
# from datetime import datetime
import datetime
import dateutil.relativedelta
from base64 import b64encode, b64decode
#  Plivo credentials
# import plivo
from random import randint
from base64 import b64decode, b64encode
from django.core.files.base import ContentFile
from django.db.models import Sum, Max
from pytz import timezone
from django.db.models import Sum
from calendar import monthrange, month_name

# canvas
import os
from reportlab.pdfgen import canvas
from reportlab.platypus import BaseDocTemplate, SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.colors import HexColor
from main.temp_pdf_generate import generate_pdf_for_merging_temp_with_main
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.display import HTML

### Total counter sale for date

In [18]:
counter_id = 9
date = '2020-12-11'

if counter_id == 'online':
    ws_sale_group_obj = SaleGroup.objects.filter(date=date, ordered_via_id__in=[1, 3],
                                                 business__business_type_id=9).order_by('id')
    ns_sale_group_obj = SaleGroup.objects.filter(date=date, ordered_via_id__in=[1, 3]).exclude(
        business__business_type_id=9).order_by('id')       
    counter_name = 'Online'
else:
    employee_trace_ids = list(
        CounterEmployeeTraceMap.objects.filter(counter_id=counter_id, collection_date='2020-12-10', counteremployeetracesalegroupmap__sale_group__date=date).values_list('id',
                                                                                                           flat=True))
    employee_trace_sale_group_obj = list(
        CounterEmployeeTraceSaleGroupMap.objects.filter(
            counter_employee_trace_id__in=employee_trace_ids).values_list(
            'sale_group', flat=True))
    ws_sale_group_obj = SaleGroup.objects.filter(id__in=employee_trace_sale_group_obj,
                                                 business__business_type_id=9).order_by('id')
    ns_sale_group_obj = SaleGroup.objects.filter(id__in=employee_trace_sale_group_obj).exclude(
        business__business_type_id=9).order_by('id')
    counter_name = Counter.objects.get(id=counter_id).name


ws_sale_obj = Sale.objects.filter(sale_group_id__in=ws_sale_group_obj, product__is_active=True).order_by(
    'sale_group_id')
ns_sale_obj = Sale.objects.filter(sale_group_id__in=ns_sale_group_obj, product__is_active=True).order_by(
    'sale_group_id')

products = Product.objects.filter(is_active=True).order_by('display_ordinal')

zone_dict = {}
for zone in Zone.objects.all().exclude(id__in=[11,12,13,15]):
    if zone.name not in zone_dict:
        zone_dict[zone.name] = {
            "amount":0,
            "order_count":[],
        }
if counter_id != 'online':
    for sale_group in ws_sale_group_obj:
        if sale_group.zone.name in zone_dict:
            zone_dict[sale_group.zone.name]['order_count'].append(sale_group.business_id)
            zone_dict[sale_group.zone.name]['amount'] += sale_group.total_cost
            
total_count = 0
for sale_group in ns_sale_group_obj:
    if sale_group.zone.name in zone_dict:
        zone_dict[sale_group.zone.name]['order_count'].append(sale_group.business_id)
        zone_dict[sale_group.zone.name]['amount'] += sale_group.total_cost
        total_count += sale_group.total_cost
print(zone_dict)
print(total_count)


if counter_id == "online":
    data_dict ={
        "counter_details": {
            "counter_id": counter_id,
            "counter_name": counter_name,
            "employee_name": [],
            "date": date
        },
        "cash":{
            "grand_total": 0,
            "booth_sale": {},
            "product_total_count": {}
        },
        "cridet":{
            "grand_total": 0,
            "booth_sale": {},
            "product_total_count": {}
        }
    }
else:
    data_dict = {
        "grand_total": 0,
        "counter_details": {
            "counter_id": counter_id,
            "counter_name": counter_name,
            "employee_name": [],
            "date": date
        },
        "booth_sale": {},
        "product_total_count": {}
    }
if counter_id == 'online':
    data_dict["counter_details"]["employee_name"].append('-')
else:
    for employee_trace_id in employee_trace_ids:
        employee_name = CounterEmployeeTraceMap.objects.get(
            id=employee_trace_id).employee.user_profile.user.first_name
        data_dict["counter_details"]["employee_name"].append(employee_name)

if counter_id == 'online':
    for product in products:
        if not product.id in data_dict["cash"]["product_total_count"]:
            data_dict["cash"]["product_total_count"][product.id] = {
                "count": 0,
                "ns_litre": 0,
                "ns_cost": 0,
                "ns_count": 0,
                "ws_litre": 0,
                "ws_cost": 0,
                "ws_count": 0,
                "total_liter": 0
            }
        if not product.id in data_dict["cridet"]["product_total_count"]:
            data_dict["cridet"]["product_total_count"][product.id] = {
                "count": 0,
                "ns_litre": 0,
                "ns_cost": 0,
                "ns_count": 0,
                "ws_litre": 0,
                "ws_cost": 0,
                "ws_count": 0,
                "total_liter": 0
            }
else:
    for product in products:
        if not product.id in data_dict["product_total_count"]:
                data_dict["product_total_count"][product.id] = {
                    "count": 0,
                    "ns_litre": 0,
                    "ns_cost": 0,
                    "ns_count": 0,
                    "ws_litre": 0,
                    "ws_cost": 0,
                    "ws_count": 0,
                    "total_liter": 0
                }

for ns in ns_sale_group_obj:
    if counter_id == "online":
        ns_sale = Sale.objects.filter(sale_group_id=ns.id, product__is_active=True)
        if ns.business.business_type_id == 1 or ns.business.business_type_id == 2:
            time = ns.time_created
            time_now = time.astimezone(timezone('Asia/Kolkata'))
            time_created = time_now.strftime("%I:%M")
            if not ns.business.code in data_dict["cash"]["booth_sale"]:
                data_dict["cash"]["booth_sale"][ns.business.code] = {
                    "date":ns.date.strftime("%d-%b"),
                    "time": time_created,
                    "zone": ns.business.zone.name,
                    "agent_name": str(BusinessAgentMap.objects.get(
                        business_id=ns.business.id).agent.first_name + " "+ BusinessAgentMap.objects.get(
                        business_id=ns.business.id).agent.last_name)[:5],
                    "product_total_cost": 0,
                    "ns": {},
                    "ws": {}
                }
        else:
            time = ns.time_created
            time_now = time.astimezone(timezone('Asia/Kolkata'))
            time_created = time_now.strftime("%I:%M")
            if not ns.business.code in data_dict["cridet"]["booth_sale"]:
                data_dict["cridet"]["booth_sale"][ns.business.code] = {
                    "date":ns.date.strftime("%d-%b"),
                    "time": time_created,
                    "zone": ns.business.zone.name,
                    "agent_name": str(BusinessAgentMap.objects.get(business_id=ns.business.id).agent.first_name+" "+BusinessAgentMap.objects.get(business_id=ns.business.id).agent.last_name)[:6],
                    "product_total_cost": 0,
                    "ns": {},
                    "ws": {}
                }

        for sale in ns_sale:
            if sale.sale_group.business.business_type_id == 1 or sale.sale_group.business.business_type_id == 2:
                for product in products:
                    if not product.id in data_dict["cash"]["booth_sale"][ns.business.code]["ns"]:
                        data_dict["cash"]["booth_sale"][ns.business.code]["ns"][product.id] = {
                            "count": 0,
                            "cost": 0
                        }

                data_dict["cash"]["booth_sale"][ns.business.code]["ns"][sale.product_id]["count"] += sale.count
                data_dict["cash"]["booth_sale"][ns.business.code]["ns"][sale.product_id]["cost"] += sale.cost
                data_dict["cash"]["booth_sale"][ns.business.code]["product_total_cost"] += sale.cost

                data_dict["cash"]["product_total_count"][sale.product_id]["count"] += sale.count
                data_dict["cash"]["product_total_count"][sale.product_id]["ns_count"] += sale.count
                data_dict["cash"]["product_total_count"][sale.product_id]["ns_litre"] += sale.count * float(
                    sale.product.quantity / 1000)
                data_dict["cash"]["product_total_count"][sale.product_id]["ns_cost"] += sale.cost
                data_dict["cash"]["product_total_count"][sale.product_id]["total_liter"] += sale.count * float(
                    sale.product.quantity / 1000)

                data_dict["cash"]["grand_total"] += sale.cost
            else:
                for product in products:
                    if not product.id in data_dict["cridet"]["booth_sale"][ns.business.code]["ns"]:
                        data_dict["cridet"]["booth_sale"][ns.business.code]["ns"][product.id] = {
                            "count": 0,
                            "cost": 0
                        }

                data_dict["cridet"]["booth_sale"][ns.business.code]["ns"][sale.product_id]["count"] += sale.count
                data_dict["cridet"]["booth_sale"][ns.business.code]["ns"][sale.product_id]["cost"] += sale.cost
                data_dict["cridet"]["booth_sale"][ns.business.code]["product_total_cost"] += sale.cost

                data_dict["cridet"]["product_total_count"][sale.product_id]["count"] += sale.count
                data_dict["cridet"]["product_total_count"][sale.product_id]["ns_count"] += sale.count
                data_dict["cridet"]["product_total_count"][sale.product_id]["ns_litre"] += sale.count * float(
                    sale.product.quantity / 1000)
                data_dict["cridet"]["product_total_count"][sale.product_id]["ns_cost"] += sale.cost
                data_dict["cridet"]["product_total_count"][sale.product_id]["total_liter"] += sale.count * float(
                    sale.product.quantity / 1000)

                data_dict["cridet"]["grand_total"] += sale.cost

    else:
        ns_sale = Sale.objects.filter(sale_group_id=ns.id, product__is_active=True)
        time = ns.time_created
        time_now = time.astimezone(timezone('Asia/Kolkata'))
        time_created = time_now.strftime("%I:%M")
        if not ns.business.code in data_dict["booth_sale"]:
            data_dict["booth_sale"][ns.business.code] = {
                "date":ns.date.strftime("%d-%b"),
                "time": time_created,
                "zone": ns.business.zone.name,
                "agent_name":str(BusinessAgentMap.objects.get(business_id=ns.business.id).agent.first_name + " " + BusinessAgentMap.objects.get(business_id=ns.business.id).agent.last_name)[:5],
                "product_total_cost": 0,
                "ns": {},
                "ws": {}
            }

        for sale in ns_sale:
            for product in products:
                if not product.id in data_dict["booth_sale"][ns.business.code]["ns"]:
                    data_dict["booth_sale"][ns.business.code]["ns"][product.id] = {
                        "count": 0,
                        "cost": 0
                    }

            data_dict["booth_sale"][ns.business.code]["ns"][sale.product_id]["count"] += sale.count
            data_dict["booth_sale"][ns.business.code]["ns"][sale.product_id]["cost"] += sale.cost
            data_dict["booth_sale"][ns.business.code]["product_total_cost"] += sale.cost

            data_dict["product_total_count"][sale.product_id]["count"] += sale.count
            data_dict["product_total_count"][sale.product_id]["ns_count"] += sale.count
            data_dict["product_total_count"][sale.product_id]["ns_litre"] += sale.count * float(
                sale.product.quantity / 1000)
            data_dict["product_total_count"][sale.product_id]["ns_cost"] += sale.cost
            data_dict["product_total_count"][sale.product_id]["total_liter"] += sale.count * float(
                sale.product.quantity / 1000)

            data_dict["grand_total"] += sale.cost

for ws in ws_sale_group_obj:
    if counter_id == "online":
        ws_sale = Sale.objects.filter(sale_group_id=ws.id, product__is_active=True)
        if ws.business.business_type_id == 1 or ws.business.business_type_id == 2 or ws.business.business_type_id == 9 :
            time = ws.time_created
            time_now = time.astimezone(timezone('Asia/Kolkata'))
            time_created = time_now.strftime("%I:%M")
            if not ws.business.code in data_dict["cash"]["booth_sale"]:
                data_dict["cash"]["booth_sale"][ws.business.code] = {
                    "date":ws.date.strftime("%d-%b"),
                    "time": time_created,
                    "zone": ws.business.zone.name,
                    "agent_name": str(BusinessAgentMap.objects.get(business_id=ws.business.id).agent.first_name + " " + BusinessAgentMap.objects.get(business_id=ws.business.id).agent.last_name)[:5],
                    "product_total_cost": 0,
                    "ns": {},
                    "ws": {}
                }
        else:
            time = ws.time_created
            time_now = time.astimezone(timezone('Asia/Kolkata'))
            time_created = time_now.strftime("%I:%M")
            if not ws.business.code in data_dict["cridet"]["booth_sale"]:
                data_dict["cridet"]["booth_sale"][ws.business.code] = {
                    "date":ws.date.strftime("%d-%b"),
                    "time": time_created,
                    "zone": ws.business.zone.name,
                    "agent_name": str(BusinessAgentMap.objects.get(business_id=ws.business.id).agent.first_name + " " + BusinessAgentMap.objects.get(business_id=ws.business.id).agent.last_name)[:5],
                    "product_total_cost": 0,
                    "ns": {},
                    "ws": {}
                }

        for sale in ws_sale:
            if sale.sale_group.business.business_type_id == 1 or sale.sale_group.business.business_type_id == 2 or ws.business.business_type_id == 9 :
                for product in products:
                    if not product.id in data_dict["cash"]["booth_sale"][ws.business.code]["ws"]:
                        data_dict["cash"]["booth_sale"][ws.business.code]["ws"][product.id] = {
                            "count": 0,
                            "cost": 0
                        }

                data_dict["cash"]["booth_sale"][ws.business.code]["ws"][sale.product_id]["count"] += sale.count
                data_dict["cash"]["booth_sale"][ws.business.code]["ws"][sale.product_id]["cost"] += sale.cost
                data_dict["cash"]["booth_sale"][ws.business.code]["product_total_cost"] += sale.cost

                data_dict["cash"]["product_total_count"][sale.product_id]["count"] += sale.count
                data_dict["cash"]["product_total_count"][sale.product_id]["ws_count"] += sale.count
                data_dict["cash"]["product_total_count"][sale.product_id]["ws_litre"] += sale.count * float(
                    sale.product.quantity / 1000)
                data_dict["cash"]["product_total_count"][sale.product_id]["ws_cost"] += sale.cost
                data_dict["cash"]["product_total_count"][sale.product_id]["total_liter"] += sale.count * float(
                    sale.product.quantity / 1000)

                data_dict["cash"]["grand_total"] += sale.cost
#             else:
#                 for product in products:
#                     if not product.id in data_dict["cridet"]["booth_sale"][ws.business.code]["ws"]:
#                         data_dict["cridet"]["booth_sale"][ws.business.code]["ws"][product.id] = {
#                             "count": 0,
#                             "cost": 0
#                         }

#                 data_dict["cridet"]["booth_sale"][ws.business.code]["ws"][sale.product_id]["count"] += sale.count
#                 data_dict["cridet"]["booth_sale"][ws.business.code]["ws"][sale.product_id]["cost"] += sale.cost
#                 data_dict["cridet"]["booth_sale"][ws.business.code]["product_total_cost"] += sale.cost

#                 data_dict["cridet"]["product_total_count"][sale.product_id]["count"] += sale.count
#                 data_dict["cridet"]["product_total_count"][sale.product_id]["ws_count"] += sale.count
#                 data_dict["cridet"]["product_total_count"][sale.product_id]["ws_litre"] += sale.count * float(
#                     sale.product.quantity / 1000)
#                 data_dict["cridet"]["product_total_count"][sale.product_id]["ws_cost"] += sale.cost
#                 data_dict["cridet"]["product_total_count"][sale.product_id]["total_liter"] += sale.count * float(
#                     sale.product.quantity / 1000)

#                 data_dict["cridet"]["grand_total"] += sale.cost


    else:
        ws_sale = Sale.objects.filter(sale_group_id=ws.id, product__is_active=True)
        time = ws.time_created
        time_now = time.astimezone(timezone('Asia/Kolkata'))
        time_created = time_now.strftime("%I:%M")
        if not ws.business.code in data_dict["booth_sale"]:
            data_dict["booth_sale"][ws.business.code] = {
                "date":ws.date.strftime("%d-%b"),
                "time": time_created,
                "zone": ws.business.zone.name,
                "agent_name": str(BusinessAgentMap.objects.get(business_id=ws.business.id).agent.first_name + " " + BusinessAgentMap.objects.get(business_id=ws.business.id).agent.last_name)[:5],
                "product_total_cost": 0,
                "ns": {},
                "ws": {}
            } 


        for sale in ws_sale:
            for product in products:
                if not product.id in data_dict["booth_sale"][ws.business.code]["ws"]:
                    data_dict["booth_sale"][ws.business.code]["ws"][product.id] = {
                        "count": 0,
                        "cost": 0
                    }

            data_dict["booth_sale"][ws.business.code]["ws"][sale.product_id]["count"] += sale.count
            data_dict["booth_sale"][ws.business.code]["ws"][sale.product_id]["cost"] += sale.cost
            data_dict["booth_sale"][ws.business.code]["product_total_cost"] += sale.cost

            data_dict["product_total_count"][sale.product_id]["count"] += sale.count
            data_dict["product_total_count"][sale.product_id]["ws_count"] += sale.count
            data_dict["product_total_count"][sale.product_id]["ws_litre"] += sale.count * float(
                sale.product.quantity / 1000)
            data_dict["product_total_count"][sale.product_id]["ws_cost"] += sale.cost
            data_dict["product_total_count"][sale.product_id]["total_liter"] += sale.count * float(
                sale.product.quantity / 1000)

            data_dict["grand_total"] += sale.cost

# data_dict['user_name'] = user_name
#     data = generate_sale_pdf_for_agent_order(data_dict, date,zone_dict)
data_dict

{'EAST': {'amount': 0, 'order_count': []}, 'WEST': {'amount': Decimal('3420.00'), 'order_count': [2766]}, 'SOUTH': {'amount': 0, 'order_count': []}, 'POLLACHI': {'amount': 0, 'order_count': []}, 'NORTH': {'amount': 0, 'order_count': []}, 'MTP': {'amount': Decimal('417125.00'), 'order_count': [2513, 2831, 2473, 2458, 2585, 2461, 2471, 2475, 2483, 2483, 2492, 2771, 2516, 2444, 2444, 2586, 2477, 2520, 2439, 2504, 2504, 2448, 2448, 2508, 2466, 2514, 2443, 2501, 2511, 2511, 2698, 2451, 2451, 2440, 2438, 2472, 2472, 2704, 2509, 2496, 2505, 2463, 2474, 2474, 2522, 2522, 2494, 2494, 2478, 2500, 2453, 2453, 2498, 2468, 2468, 2447, 2519, 2454, 2744, 2490, 2584, 2584, 2450, 2450, 2699, 2699, 2605, 2605, 2445, 2445, 2467, 2467]}, 'CURD Zone': {'amount': 0, 'order_count': []}}
417125.00


{'grand_total': Decimal('420545.00'),
 'counter_details': {'counter_id': 9,
  'counter_name': 'Mettupaalayam',
  'employee_name': ['Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy',
   'Thirumoorthy'

In [8]:
employee_trace_ids = list(CounterEmployeeTraceMap.objects.filter(counter_id=9, counteremployeetracesalegroupmap__sale_group__date=date).values_list('id', 'counteremployeetracesalegroupmap__sale_group'))      
len(employee_trace_ids)

74

In [9]:
employee_trace_sale_group_obj = list(
        CounterEmployeeTraceSaleGroupMap.objects.filter(
            counter_employee_trace_id__in=employee_trace_ids).values_list(
            'sale_group', flat=True))

147

841090